## AKI 예측 모델
- 보호변수는 학습에 빼고

In [150]:
import pandas as pd
train2 = pd.read_csv('checkpoint2_trian.csv')
test2 = pd.read_csv('checkpoint2_test.csv') # 이거 다시 valid test 로 나눠야함
train2.columns

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', '최초작성일자', '수진일자', '수진 당시 나이',
       '입원일자', '퇴원일자', '입원시 진료과', '기저크레아티닌', '기저eGFR', '중환자실', '입원 일수',
       'ori_target', '성별', 'diagnose', '수진일 당시 직업', '종교', '수진일 당시 결혼상태',
       '음주기간(년)', 'medicine', 'sepsis', 'AKI', 'height', 'weight'],
      dtype='object')

In [151]:
# '최초작성일자','수진일자',   '입원일자', '퇴원일자', '입원 일수', 
# 'ori_target' -> 예측할 AKi 여부는 우리가 직접 정한 기준으로 간다
# 위에 칼럼들 다 버리자 
drop_col = ['최초작성일자','수진일자',   '입원일자', '퇴원일자', '입원 일수', 'ori_target']
train2.drop( columns = drop_col, axis = 1, inplace=True)
test2.drop( columns = drop_col, axis = 1, inplace=True)

슬라이딩 윈도우 적용할라고 일단 환자마다 1일치를 복붙했는데 이거 채워넣어야함

In [153]:
df=train2.copy()
df['간호기록 작성일자'] = pd.to_datetime(df['간호기록 작성일자'])  # 날짜 형식 변환
first_day_data = df.groupby('등록번호').first().reset_index()
first_day_data['간호기록 작성일자'] = first_day_data['간호기록 작성일자'] - pd.Timedelta(days=1)
first_day_data['AKI'] = 0

# 원본 데이터에 추가
df_extended = pd.concat([first_day_data, df], ignore_index=True)

# 결과 정렬
train = df_extended.sort_values(by=['등록번호', '간호기록 작성일자']).reset_index(drop=True)

df=test2.copy()
df['간호기록 작성일자'] = pd.to_datetime(df['간호기록 작성일자'])  # 날짜 형식 변환
first_day_data = df.groupby('등록번호').first().reset_index()
first_day_data['간호기록 작성일자'] = first_day_data['간호기록 작성일자'] - pd.Timedelta(days=1)
first_day_data['AKI'] = 0

# 원본 데이터에 추가
df_extended = pd.concat([first_day_data, df], ignore_index=True)

# 결과 정렬
test = df_extended.sort_values(by=['등록번호', '간호기록 작성일자']).reset_index(drop=True)

In [154]:
train2.columns

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', '수진 당시 나이', '입원시 진료과', '기저크레아티닌',
       '기저eGFR', '중환자실', '성별', 'diagnose', '수진일 당시 직업', '종교', '수진일 당시 결혼상태',
       '음주기간(년)', 'medicine', 'sepsis', 'AKI', 'height', 'weight'],
      dtype='object')

In [155]:
set(train2.성별)

{0, 1}

In [156]:
# 데이터에 1대1 매핑되는 칼럼들 merge해줄거임)
hadm = ['등록번호']
onebyone = ['기저크레아티닌','중환자실','diagnose', '음주기간(년)','weight', 'height','medicine', 'sepsis','기저eGFR']
hadm.extend(onebyone)
# 보호변수 (성별, 연령, 직업, 종교, 인종, 진료과 등등)
protect_variable = ['성별','종교','수진일 당시 직업','수진 당시 나이','수진일 당시 결혼상태', '입원시 진료과']
hadm.extend(protect_variable)
d_onebyone_table_train = train2[hadm]
d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_train

d_onebyone_table_test = test2[hadm]
d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_test

/tmp/ipykernel_40820/3494105275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
/tmp/ipykernel_40820/3494105275.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,등록번호,기저크레아티닌,중환자실,diagnose,음주기간(년),weight,height,medicine,sepsis,기저eGFR,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,30,0.97,0.0,0,0.0,56.10,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
7,142,1.21,1.0,0,0.0,57.00,168.0,1,0,80.415358,0,무교,회사원,29.0,미혼,신경외과
15,150,0.61,0.0,0,0.0,45.59,149.0,1,0,98.533870,1,불교,주부,60.0,기혼,신경외과 (뇌신경센터)
23,195,0.55,0.0,0,0.0,62.20,156.9,1,0,110.138206,1,무교,기타,49.0,기혼,신경외과 (뇌신경센터)
31,322,0.86,0.0,0,0.0,47.45,155.3,1,0,65.618255,1,불교,무직,76.0,사별,종합내과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24568,229209,0.83,0.0,0,50.0,68.40,171.4,1,0,91.048194,0,무교,자유업,67.0,기혼,응급의학과
24575,229210,0.89,0.0,0,60.0,53.45,163.4,1,0,83.053201,0,무교,무직,76.0,기혼,비뇨의학과
24582,229237,0.78,0.0,0,0.0,67.50,157.2,3,0,75.944082,1,개신교,주부,72.0,기혼,뇌신경센터
24590,229304,1.34,0.0,3,55.0,78.00,175.0,1,0,50.381372,0,무교,기타,78.0,기혼,소화기센터


In [157]:
train2.drop(columns = onebyone,axis =1 , inplace = True)
test2.drop(columns = onebyone,axis =1 , inplace = True)

# 보호변수도 학습에 사용안할거임
train2.drop(columns = protect_variable, axis = 1, inplace = True)
test2.drop(columns = protect_variable, axis = 1, inplace = True)


In [158]:
print(train2.columns, len(train2.columns))
print(test2.columns)

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', 'AKI'],
      dtype='object') 28
Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', 'AKI'],
      dtype='object')


In [159]:
import pandas as pd
import numpy as np

# 슬라이딩 윈도우 생성 함수
def create_sliding_windows_as_df(data, time_steps=2):
    X, y = [], []
    columns = data.drop(columns=['AKI']).columns
    
    for i in range(len(data) - time_steps):
        # X: 현재 타임스텝에 해당하는 윈도우 데이터 (time_steps * 특성 수 크기)
        window_data = data.iloc[i:i + time_steps].drop(columns=['AKI']).values.flatten()
        X.append(window_data)
        
        # y: 현재 윈도우 끝난 후 48시간 이내의 AKI 여부
        y_values = data.iloc[i + time_steps:i + time_steps + 2]['AKI'].values
        y.append(1 if 1 in y_values else 0)
    
    # 컬럼 이름 만들기 (time_steps별로 컬럼에 번호 붙이기)
    flattened_columns = [f"{col}_t{t}" for t in range(1, time_steps + 1) for col in columns]
    
    # X를 데이터프레임으로 변환
    X_df = pd.DataFrame(X, columns=flattened_columns)
    y_df = pd.Series(y, name='AKI')
    
    return X_df, y_df

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in train2.groupby('등록번호'):
    group = group.sort_values(by='간호기록 작성일자')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
train2_final = pd.concat([X_all, y_all], axis=1)
train2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in test2.groupby('등록번호'):
    group = group.sort_values(by='간호기록 작성일자')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
test2_final = pd.concat([X_all, y_all], axis=1)
test2_final.head()  # 최종 데이터프레임 확인

(160064, 54)
(160064,)
(17706, 54)
(17706,)


,등록번호_t1,간호기록 작성일자_t1,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,...,"Bilirubin, total_t2","CO2, total (serum)_t2","Calcium, total_t2",Chloride (serum)_t2,Phosphorus_t2,Potassium(serum)_t2,Sodium(serum)_t2,WBC_t2,Creatinine_t2,AKI
0,30,2022-05-25,36.0,36.6,36.300000,65.0,67.0,66.000000,79.0,87.0,...,1.040,27.0,8.35,105.0,3.05,4.15,140.5,16.34,1.010000,0
1,30,2022-05-26,35.3,36.7,36.163636,49.0,117.0,69.550000,81.0,81.0,...,0.825,27.0,8.05,104.0,2.80,4.50,138.0,9.71,1.133333,0
2,30,2022-05-27,36.3,37.3,36.813333,49.0,64.0,55.600000,92.0,92.0,...,1.210,27.0,7.85,106.0,1.75,4.35,139.5,9.94,0.883333,0
3,30,2022-05-28,36.8,37.3,37.066667,59.0,70.0,63.833333,103.0,103.0,...,1.420,27.0,8.20,103.0,2.30,5.00,139.0,10.30,0.790000,0
4,30,2022-05-29,36.7,36.9,36.800000,62.0,71.0,66.000000,103.0,103.0,...,2.040,27.0,8.90,104.0,2.60,4.40,139.0,8.82,0.770000,0


In [161]:
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('간호기록 작성일자')]]
train2_final.columns[train2_final.columns.str.contains('등록번호')][1:]
train2_final.drop(columns=list(train2_final.columns[train2_final.columns.str.contains('등록번호')][1:]),
                  axis = 1, inplace=True)
# train2_final
train2_final.rename(columns={'등록번호_t1': '등록번호'}, inplace=True)
train2_final = pd.merge(train2_final,d_onebyone_table_train,how='left',on='등록번호')
train2_final

test2_final = test2_final[test2_final.columns[~test2_final.columns.str.contains('간호기록 작성일자')]]
test2_final.columns[test2_final.columns.str.contains('등록번호')][1:]
test2_final.drop(columns=list(test2_final.columns[test2_final.columns.str.contains('등록번호')][1:]),
                  axis = 1, inplace=True)

test2_final.rename(columns={'등록번호_t1': '등록번호'}, inplace=True)
test2_final = pd.merge(test2_final,d_onebyone_table_test,how='left',on='등록번호')
test2_final

,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,height,medicine,sepsis,기저eGFR,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,30,36.0,36.6,36.300000,65.0,67.0,66.000000,79.0,87.0,84.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
1,30,35.3,36.7,36.163636,49.0,117.0,69.550000,81.0,81.0,81.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
2,30,36.3,37.3,36.813333,49.0,64.0,55.600000,92.0,92.0,92.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
3,30,36.8,37.3,37.066667,59.0,70.0,63.833333,103.0,103.0,103.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
4,30,36.7,36.9,36.800000,62.0,71.0,66.000000,103.0,103.0,103.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17701,229356,36.2,36.7,36.533333,73.0,78.0,76.000000,61.0,69.0,64.750000,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17702,229356,36.6,36.8,36.700000,68.0,76.0,72.666667,69.0,81.0,73.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17703,229356,36.2,37.4,36.866667,74.0,80.0,76.333333,68.0,75.0,72.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17704,229356,35.9,37.4,36.226190,46.0,95.0,73.476190,71.0,88.0,78.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과


In [162]:
protect_variable

['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

In [163]:
# train2_final.drop(columns = protect_variable, axis = 1, inplace = True)
# test2_final.drop(columns = protect_variable, axis = 1, inplace = True)
train2_final.drop(columns =['기저eGFR'], axis = 1, inplace = True)
test2_final.drop(columns = ['기저eGFR'], axis = 1, inplace = True)
# # 보호변수 제거나중에 해야함

In [164]:
train2_final.head()


,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,weight,height,medicine,sepsis,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,9,36.0,37.0,36.400000,35.0,67.0,50.545455,56.0,60.0,57.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
1,9,36.3,36.6,36.433333,42.0,74.0,56.100000,54.0,58.0,55.750000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
2,9,36.3,37.1,36.675000,61.0,73.0,66.400000,54.0,82.0,65.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
3,9,36.4,36.9,36.700000,66.0,91.0,74.000000,59.0,69.0,63.000000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
4,9,36.7,36.9,36.766667,67.0,87.0,74.000000,65.0,83.0,74.000000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터


### 스케일링전 checkpoint3

In [ ]:
# train2_final.to_csv('checkpoint3_train.csv',index = False)
# test2_final.to_csv('checkpoint3_test.csv',index = False)x

In [173]:
train2_final = pd.read_csv('checkpoint3_train.csv')
test2_final = pd.read_csv('checkpoint3_test.csv')

In [174]:
train2_final.head(2)

,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,weight,height,medicine,sepsis,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,9,36.0,37.0,36.400000,35.0,67.0,50.545455,56.0,60.0,57.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
1,9,36.3,36.6,36.433333,42.0,74.0,56.100000,54.0,58.0,55.750000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터


In [175]:
train2_final.columns

Index(['등록번호', 'BT_min_t1', 'BT_max_t1', 'BT_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'PR_min_t1', 'PR_max_t1', 'PR_mean_t1',
       'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'ALT(GPT)_t1', 'AST(GOT)_t1',
       'Albumin_t1', 'Alkaline Phosphatase_t1', 'Bilirubin, total_t1',
       'CO2, total (serum)_t1', 'Calcium, total_t1', 'Chloride (serum)_t1',
       'Phosphorus_t1', 'Potassium(serum)_t1', 'Sodium(serum)_t1', 'WBC_t1',
       'Creatinine_t1', 'BT_min_t2', 'BT_max_t2', 'BT_mean_t2', 'DBP_min_t2',
       'DBP_max_t2', 'DBP_mean_t2', 'PR_min_t2', 'PR_max_t2', 'PR_mean_t2',
       'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2', 'ALT(GPT)_t2', 'AST(GOT)_t2',
       'Albumin_t2', 'Alkaline Phosphatase_t2', 'Bilirubin, total_t2',
       'CO2, total (serum)_t2', 'Calcium, total_t2', 'Chloride (serum)_t2',
       'Phosphorus_t2', 'Potassium(serum)_t2', 'Sodium(serum)_t2', 'WBC_t2',
       'Creatinine_t2', 'AKI', '기저크레아티닌', '중환자실', 'diagnose', '음주기간(년)',
       'weight', 'he

### standard 스케일링

In [176]:
from sklearn.model_selection import train_test_split

# 고유한 등록번호 추출
unique_ids = train2_final['등록번호'].unique()

# 등록번호 기준으로 80%를 학습, 10%를 검증 데이터셋으로 나누기
train_ids, valid_ids = train_test_split(unique_ids, test_size=0.1, random_state=42)

# 학습 데이터와 검증 데이터 생성
train_df = train2_final[train2_final['등록번호'].isin(train_ids)]
valid_df = train2_final[train2_final['등록번호'].isin(valid_ids)]
test_df = test2_final.copy()
# 결과 확인
print(f"Train Data Size: {len(train_df)}, Valid Data Size: {len(valid_df)}, Test Data Size : {len(test_df)}")

Train Data Size: 144115, Valid Data Size: 15989, Test Data Size : 17706


In [177]:
set(valid_ids).intersection(train_ids)

set()

In [ ]:
num_col_to_scale = ['weight', 'height','기저크레아티닌','음주기간(년)'] + [col for col in train2_final.columns if 't1' in col or 't2' in col]
cat_col = ['medicine','diagnose','중환자실','sepsis']
target_col = ['AKI']

from sklearn.preprocessing import StandardScaler

# 스케일러 인스턴스 생성
scaler = StandardScaler()
train_df[num_col_to_scale] = scaler.fit_transform(train_df[num_col_to_scale])

# 테스트 데이터 스케일링
# 주의: 테스트 데이터는 'fit'하지 않고 'transform'만 실행
valid_df[num_col_to_scale] = scaler.transform(valid_df[num_col_to_scale])
test_df[num_col_to_scale] = scaler.transform(test_df[num_col_to_scale])

- 범주형 변수 인코딩
    - medicine 칼럼 : {1, 2, 3}로 이루어져 있음 -> 순위가 있는 범주형이라 따로 인코딩 하지 않는다
    - diagnose 칼럼 : 0,1,2,3으로 이루어져 있음 -> 원핫인코딩 : 범주가 순서를 가지는 변수가 아니라서 원핫인코딩 선택
    - 중환자실 칼럼 : {0.0, 1.0} -> 따로 인코딩 x 
    - sepsis : {0,1} -> 따로인코딩 x

In [179]:
# set(train2_final[cat_col[0]]) # {1, 2, 3}
# set(train2_final[cat_col[1]]) # {0, 1, 2, 3}
# set(train2_final[cat_col[2]]) # {0.0, 1.0}
set(train2_final[cat_col[3]]) #  {0, 1}

{0, 1}

In [ ]:
train_df['중환자실'] = train_df['중환자실'].astype(int)
valid_df['중환자실'] = valid_df['중환자실'].astype(int)
test_df['중환자실'] = test_df['중환자실'].astype(int)

In [181]:
# One-Hot Encoding for 'diagnose' (순서가 없는 다중 범주형 변수)
train_df = pd.get_dummies(train_df, columns=['diagnose'], prefix='diag')
valid_df = pd.get_dummies(valid_df, columns=['diagnose'], prefix='diag')
test_df = pd.get_dummies(test_df, columns=['diagnose'], prefix='diag')

# train, valid, test 데이터셋 간의 칼럼 수를 맞추기 위해 align 사용
train_df, valid_df = train_df.align(valid_df, join='outer', axis=1, fill_value=0)
train_df, test_df = train_df.align(test_df, join='outer', axis=1, fill_value=0)
valid_df, test_df = valid_df.align(test_df, join='outer', axis=1, fill_value=0)

# 인코딩된 'diag' 칼럼들을 정수형(int)으로 변환
diag_columns = [col for col in train_df.columns if col.startswith('diag_')]
train_df[diag_columns] = train_df[diag_columns].astype(int)
valid_df[diag_columns] = valid_df[diag_columns].astype(int)
test_df[diag_columns] = test_df[diag_columns].astype(int)

### checkpoint4

In [ ]:
# train_df.to_csv('checkpoint4_train.csv',index = False)
# test_df.to_csv('checkpoint4_test.csv',index = False)
# valid_df.to_csv('checkpoint4_valid.csv',index = False)

In [78]:
import pandas as pd
train_df = pd.read_csv('../tabnet/checkpoint4_train.csv')
test_df = pd.read_csv('../tabnet/checkpoint4_test.csv')
valid_df = pd.read_csv('../tabnet/checkpoint4_valid.csv')

In [135]:
def filter_aki_rows(df):
    rows_to_drop = []
    
    # 등록번호별로 그룹화하여 처리
    for reg_num in df['등록번호'].unique():
        reg_num_df = df[df['등록번호'] == reg_num]
        
        # AKI가 1인 첫 번째 인덱스 찾기
        aki_indices = reg_num_df[reg_num_df['AKI'] == 1].index
        
        if len(aki_indices) > 0:
            first_aki_index = aki_indices[0]
            # 그 등록번호 내에서 AKI가 1인 시점 이후의 모든 데이터 제거
            rows_to_drop.extend(reg_num_df[reg_num_df.index > first_aki_index].index)
    
    # rows_to_drop에 해당하는 데이터만 제거
    filtered_df = df.drop(rows_to_drop)
    
    return filtered_df

# 함수 실행
train_df = filter_aki_rows(train_df)
valid_df = filter_aki_rows(valid_df)
test_df = filter_aki_rows(test_df)

In [136]:
print(f"Train Data Size: {len(train_df)}, Valid Data Size: {len(valid_df)}, Test Data Size : {len(test_df)}")

Train Data Size: 148634, Valid Data Size: 16517, Test Data Size : 18304


In [137]:
train_df.reset_index(drop = True,inplace =True)
test_df.reset_index(drop = True,inplace =True)
valid_df.reset_index(drop = True,inplace =True)

- 전처리 끝났고 진짜 모델

In [114]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [115]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression,RidgeClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.model_selection import train_test_split as tts

import random
import os

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import warnings
import pandas as pd
import matplotlib.pyplot as plt

os.environ['KMP_DUPLICATE_LIB_OK']='True'
warnings.filterwarnings("ignore", category=FutureWarning)

- reweighing 전 데이터로 예측한 모델 결과로 보호변수별로 범주별로 EOD와 DI를 계산한다 -> 이때 나온 EOD와 DI 점수를 가지고 reweighing 의 특권 그룹과 비특권 그룹을 결정하고 그걸 기반으로 데이터 전처리 
- 특권 그룸 :긍정적인 예측, 질병이 없다고 예측을 더 많이 받을 것으로 기대되는 그룹
- 비특권 그룹 : 그 반대

In [116]:
protect_variable = ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']
등록번호_보호변수 = ['등록번호'] + protect_variable
등록번호_보호변수

['등록번호', '성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

In [117]:
set(train_df['수진일 당시 직업'])

{'0', '군인', '기타', '무직', '자유업', '전문직', '주부', '학생', '회사원'}

In [118]:
protect_variable = ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']
등록번호_보호변수 = ['등록번호'] + protect_variable
등록번호_보호변수

print(protect_variable)
# for i in protect_variable:
#     print('\n')
#     print(test_df[i].value_counts().sort_index())
#     print('\n')
# age_group_counts = test_df['age_group_big'].value_counts().sort_index()
# print(f'\n\n{age_group_counts}')

## 나이 그룹 관련 함수들 *****************************
def 만나이플러스1(df):
    data = df
    data['수진 당시 나이+1(만나이x)'] = data['수진 당시 나이']+1
    return data
    
# 20대 30대 이렇게 구별해주는 함수 
def map_age_to_decade(age):
    return f"{(age // 10) * 10}대"
    
def classify_age_group(age):
    if age <= 29:
        return 'Young Adults'  # 청년층
    elif 30 <= age <= 64:
        return 'Middle-aged Adults'  # 중장년층
    elif age >= 65:
        return 'Older Adults'  # 노년층
    return 'Unknown'  # 예외 처리

# 입원시 진료과 그룹 줄이는 함수
# 데이터프레임이 입력이고 biggest,bigger 로 그룹화한 칼럼 붙여서 데이터 프레임 반환
def classify_department_biggest(df):
    biggest_mapping = {
    '내분비대사내과 (노인의료센터)' : '내과 계열',
    '내분비대사내과': '내과 계열',
    '내분비대사내과 (암센터)': '내과 계열',
    '노인병내과 (노인의료센터)': '내과 계열',
    '노인병내과 (심장혈관센터)': '내과 계열',
    '감염내과': '내과 계열',
    '류마티스내과': '내과 계열',
    '소화기내과': '내과 계열',
    '소화기내과 (암센터)': '내과 계열',
    '순환기내과': '내과 계열',
    '순환기내과 (심장혈관센터)': '내과 계열',
    '신경과 (뇌신경센터)': '내과 계열',
    '신장내과': '내과 계열',    
    '알레르기내과': '내과 계열',
    '종합내과': '내과 계열',
    '혈액종양내과': '내과 계열',
    '혈액종양내과 (암센터)': '내과 계열',
    '호흡기내과': '내과 계열',
    '호흡기내과 (암센터)': '내과 계열',
    '호흡기내과 (폐센터)': '내과 계열',
    '내과' : '내과 계열',
    
    '응급의학과': '응급의학과',
         
    '노인의료센터': '기타 진료과',
    '마취통증의학과': '기타 진료과',
    '소아청소년과 (내분비대사)': '기타 진료과',
    '소아청소년과 (소화기영양)': '기타 진료과',
    '소아청소년과 (심장)': '기타 진료과',
    '소아청소년과 (알레르기호흡기)': '기타 진료과',
    '안과': '기타 진료과',
    '이비인후과': '기타 진료과',
    '재활의학과': '기타 진료과',
    '피부과': '기타 진료과',
    '핵의학과 (암센터)' : '기타 진료과',
    
    '비뇨의학과': '외과 계열',
    '산부인과': '외과 계열',
    '성형외과': '외과 계열',
    '신경외과': '외과 계열', 
    '심장혈관흉부외과': '외과 계열',
    '외과': '외과 계열',
    '치과': '외과 계열',
    '관절센터': '외과 계열',
    '뇌신경센터': '외과 계열',
    '소화기센터': '내과 계열',
    '심장혈관센터': '내과 계열',            
    '척추센터': '외과 계열',
    '폐센터': '외과 계열',
    '신경외과 (뇌신경센터)': '외과 계열',
    '신경외과 (암센터)': '외과 계열',
    '신경외과 (척추센터)': '외과 계열', 
    '심장혈관흉부외과 (심장혈관센터)': '외과 계열',
    '심장혈관흉부외과 (폐센터)': '외과 계열',
     '외과 (암센터)': '외과 계열',
    '정형외과 (관절센터)': '외과 계열',
    '정형외과 (암센터)': '외과 계열',
    '정형외과 (척추센터)': '외과 계열',
    '비뇨의학과 (암센터)': '외과 계열',
     '류마티스내과 (관절센터)' :'외과 계열',
    '마취통증의학과 (암센터)' :'외과 계열' ,
    }
    df['department_group_bigggest'] = df['입원시 진료과'].replace(biggest_mapping)
    return df

    # df['department_group_bigggest'] = df['입원시 진료과'].replace(biggest_mapping)
    
    # return df
    
def classify_department_bigger(df):
    bigger_mapping ={
    '감염내과': '기타 진료과',
    '관절센터': '정형외과·재활 계열',
    '내과': '내과 계열',
    '내분비대사내과': '내과 계열',
    '내분비대사내과 (노인의료센터)': '노인·종합진료',
    '노인병내과 (노인의료센터)': '노인·종합진료',
    '노인의료센터': '노인·종합진료',
    '뇌신경센터': '신경과·뇌 관련',
    '류마티스내과': '내과 계열',
    '마취통증의학과': '기타 진료과',
    '비뇨의학과': '외과 계열',
    '비뇨의학과 (암센터)': '암 치료 관련',
    '산부인과': '외과 계열',
    '성형외과': '외과 계열',
    '소아청소년과 (알레르기호흡기)': '기타 진료과',
    '소화기내과': '소화기·순환기 계열',
    '소화기내과 (암센터)': '암 치료 관련',
    '소화기센터': '소화기·순환기 계열',
    '순환기내과 (심장혈관센터)': '소화기·순환기 계열',
    '신경과 (뇌신경센터)': '신경과·뇌 관련',
    '신경외과': '외과 계열',
    '신경외과 (뇌신경센터)': '신경과·뇌 관련',
    '신경외과 (척추센터)': '정형외과·재활 계열',
    '신장내과': '내과 계열',
    '심장혈관센터': '소화기·순환기 계열',
    '심장혈관흉부외과 (심장혈관센터)': '소화기·순환기 계열',
    '심장혈관흉부외과 (폐센터)': '소화기·순환기 계열',
    '안과': '기타 진료과',
    '알레르기내과': '내과 계열',
    '외과': '외과 계열',
    '외과 (암센터)': '암 치료 관련',
    '응급의학과': '기타 진료과',
    '이비인후과': '기타 진료과',
    '재활의학과': '정형외과·재활 계열',
    '정형외과 (관절센터)': '정형외과·재활 계열',
    '정형외과 (암센터)': '암 치료 관련',
    '정형외과 (척추센터)': '정형외과·재활 계열',
    '종합내과': '노인·종합진료',
    '척추센터': '정형외과·재활 계열',
    '치과': '외과 계열',
    '폐센터': '소화기·순환기 계열',
    '혈액종양내과': '내과 계열',
    '혈액종양내과 (암센터)': '암 치료 관련',
    '호흡기내과 (폐센터)': '내과 계열'
}
    df['department_group_bigger'] = df['입원시 진료과'].replace(bigger_mapping)
    
    return df

def classify_종교(df):
    종교_mapping ={
  '0':'기타', '개신교':'개신교', '기타':'기타', '무교':'무교', '불교':'불교', '천주교':'천주교'
}
    df['종교_group'] = df['종교'].replace(종교_mapping)
    
    return df

def classify_직업(df):
    job_mapping = {'0': '기타', '군인':'군인', 
                   '기타':'기타', '무직':'무직', '자유업':'자유업', '전문직':'전문직', 
                   '주부':'주부', '학생':'학생', '회사원':'회사원'}
    df['직업_group'] = df['수진일 당시 직업'].replace(job_mapping)
    
    return df

공정성_관련_칼럼 = ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

# 연령대 컬럼 추가(수진 당시 나이+1(만나이x))
train_df = 만나이플러스1(train_df)
test_df = 만나이플러스1(test_df)
valid_df = 만나이플러스1(valid_df)
공정성_관련_칼럼.append('수진 당시 나이+1(만나이x)')


train_df['연령대'] = train_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
test_df['연령대'] = test_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
valid_df['연령대'] = valid_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
공정성_관련_칼럼.append('연령대')


# 새로운 열 'age_group_big' 추가
train_df['age_group_big'] = train_df['수진 당시 나이+1(만나이x)'].apply(classify_age_group)
test_df['age_group_big'] = test_df['수진 당시 나이+1(만나이x)'].apply(classify_age_group)
valid_df['age_group_big'] =  valid_df['수진 당시 나이+1(만나이x)'].apply(classify_age_group)
공정성_관련_칼럼.append('age_group_big')

# department
train_df = classify_department_biggest(train_df)
test_df = classify_department_biggest(test_df)
valid_df = classify_department_biggest(valid_df)
공정성_관련_칼럼.append('department_group_bigggest')

train_df = classify_department_bigger(train_df)
test_df = classify_department_bigger(test_df)
valid_df = classify_department_bigger(valid_df)
공정성_관련_칼럼.append('department_group_bigger')

# 종교
train_df = classify_종교(train_df)
test_df = classify_종교(test_df)
valid_df = classify_종교(valid_df)
공정성_관련_칼럼.append('종교_group')

# 직업
train_df = classify_직업(train_df)
test_df = classify_직업(test_df)
valid_df = classify_직업(valid_df)
공정성_관련_칼럼.append('직업_group')


print(공정성_관련_칼럼)# ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과', '수진 당시 나이+1(만나이x)', '연령대', 'age_group_big', 'department_group_bigggest', 'department_group_bigger']
공정성_관련_칼럼 = ['성별',  
             '수진일 당시 직업', '직업_group',
             '종교','종교_group',
             '수진 당시 나이','수진 당시 나이+1(만나이x)', '연령대', 'age_group_big',
             '수진일 당시 결혼상태',
             '입원시 진료과', 'department_group_bigger' ,'department_group_bigggest']

for i in 공정성_관련_칼럼:
    print('\n')
    print(train_df[i].value_counts().sort_index())
    print('\n')

['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']
['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과', '수진 당시 나이+1(만나이x)', '연령대', 'age_group_big', 'department_group_bigggest', 'department_group_bigger', '종교_group', '직업_group']


성별
0    82630
1    66004
Name: count, dtype: int64




수진일 당시 직업
0       2461
군인       970
기타     12071
무직     59991
자유업    15514
전문직     7342
주부     31874
학생      1346
회사원    17065
Name: count, dtype: int64




직업_group
군인       970
기타     14532
무직     59991
자유업    15514
전문직     7342
주부     31874
학생      1346
회사원    17065
Name: count, dtype: int64




종교
0       2247
개신교    38037
기타      2650
무교     62841
불교     26512
천주교    16347
Name: count, dtype: int64




종교_group
개신교    38037
기타      4897
무교     62841
불교     26512
천주교    16347
Name: count, dtype: int64




수진 당시 나이
19.0     367
20.0     392
21.0     465
22.0     273
23.0     372
        ... 
86.0    2405
87.0    2200
88.0    2048
89.0    1484
90.0    4831
Name: count, Length: 72,

- 보호변수들에 대해 라벨 인코딩 수행

In [119]:
# 공정성_관련_칼럼 = ['성별',  
            #  '수진일 당시 직업', '직업_group',
            #  '종교','종교_group',
            #  '수진 당시 나이','수진 당시 나이+1(만나이x)', '연령대', 'age_group_big',
            #  '수진일 당시 결혼상태',
            #  '입원시 진료과', 'department_group_bigger' ,'department_group_bigggest']


protect_variable = ['성별',  '직업_group',
                    '종교_group',
                 '연령대',
              '수진일 당시 결혼상태',
             'department_group_bigggest']

공정성중_버릴_칼럼 =  ['수진일 당시 직업',
             '수진 당시 나이','수진 당시 나이+1(만나이x)', 'age_group_big','종교',
             '입원시 진료과', 'department_group_bigger' ]


train_df.drop(columns = 공정성중_버릴_칼럼,axis = 1,inplace=True)
test_df.drop(columns = 공정성중_버릴_칼럼,axis = 1,inplace=True)
valid_df.drop(columns = 공정성중_버릴_칼럼,axis = 1,inplace=True)

# 1. 라벨 인코딩 적용
label_enc = LabelEncoder()
label_enc_recod = {}
for var in protect_variable:
    train_df[var] = label_enc.fit_transform(train_df[var])
    valid_df[var] = label_enc.transform(valid_df[var])
    test_df[var] = label_enc.transform(test_df[var])
    label_enc_recod[var] = label_enc.classes_

# 모델 학습에 사용할 피처에서 보호변수는 제외
X_train = train_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)
X_valid = valid_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)
X_test = test_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)
# 라벨
y_train = train_df['AKI']
y_valid = valid_df['AKI']
y_test = test_df['AKI']

In [120]:

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[0, 1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[0, 1])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[0, 1])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도(precision 0.2 목표): {:.4f}".format(precision))
    print("재현율(recall 0.75 목표): {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    

# ClassificationMetric 라이브러리를 사용해 공정성 지표(EOD, DI)를 계산하는 함수 정의
from aif360.metrics import ClassificationMetric

def calculate_eod_di_by_category(dataset, dataset_pred, protected_var,
                                 label_enc_recod = label_enc_recod ,valid = False,print_ = True):
    # BinaryLabelDataset의 features에서 보호변수 추출 (정수형으로 인코딩된 값이 있을 것임)
    # 먼저 보호변수를 추출합니다. BinaryLabelDataset에서는 features 속성으로 데이터 접근 가능
    protected_idx = dataset.feature_names.index(protected_var)
    protected_values = dataset.features[:, protected_idx]
    
    # 보호변수의 범주 추출 (유니크한 값들)
    unique_categories = np.unique(protected_values)
    record = label_enc_recod[protected_var]
    
    privilege_group = {}# 라벨인코딩 값 저장하기
    nonprovilege_group = {}
    
    eod_scores = {}
    di_scores = {}
    
    for category in unique_categories:
        i = int(category)
        if print_:
            print(f"\n=== {protected_var}: {record[i]} 기준 공정성 평가 ===")
        
        # 비특권 그룹: 해당 범주를 비특권 그룹으로 설정
        unprivileged_groups = [{protected_var: category}]
        
        # 특권 그룹: 나머지 범주들을 특권 그룹으로 설정
        privileged_groups = [{protected_var: val} for val in unique_categories if val != category]
        
        # 공정성 지표 계산 (EOD와 DI)
        metric = ClassificationMetric(dataset, dataset_pred,
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)
        
        eod = metric.equal_opportunity_difference()  # Equal Opportunity Difference 계산
        di = metric.disparate_impact()  # Disparate Impact 계산
        
        # EOD와 DI 값을 저장
        name = str(protected_var) + str(record[i])
        eod_scores[name] = eod
        di_scores[name] = di
        
        if print_:
            print(f"EOD: {eod}\n(0과의 거리:{abs(eod-0)}) \n\nDI: {di}\n(1과의 거리:{abs(di-1)})\n")
        
        if valid:
            if (eod>0)&(di<1):
                nonprovilege_group.setdefault(protected_var,[])
                nonprovilege_group[protected_var].append(i)
            elif (eod<0)&(di>1):
                privilege_group.setdefault(protected_var,[])
                privilege_group[protected_var].append(i)
            else:
                if abs(eod) > abs(di-1):
                    if eod > 0 :
                        nonprovilege_group.setdefault(protected_var,[])
                        nonprovilege_group[protected_var].append(i)
                    else:
                        privilege_group.setdefault(protected_var,[])
                        privilege_group[protected_var].append(i)
                else:
                    if di < 1 :
                        nonprovilege_group.setdefault(protected_var,[])
                        nonprovilege_group[protected_var].append(i) 
                    else:
                        privilege_group.setdefault(protected_var,[])
                        privilege_group[protected_var].append(i)
    
    if valid:
        return privilege_group,nonprovilege_group
    else:
        return {},{}


# Reweighing과 하이퍼파라미터 튜닝 적용 함수
def apply_reweighing_tuning(train_dataset, valid_dataset,
                            valid_dataset_pred_rw_each, # 보호변수 하나에 대해서만 rw적용한 걸로 예측한 결과
                            # test_dataset,
                            protect_var,  # 여기서 준 protect_var,즉 보호변수에 대해서만 이전 결과의 eod id점수로 특권 비특권 나눠서 rw함
                            best_param,# rw 전 데이터로 튜닝했을때 베스트 파람
                            label_enc_recod = label_enc_recod,
                            X_train=X_train,X_valid=X_valid,X_test=X_test,y_valid=y_valid):
    # import optuna
    # 보호변수별(protect_var에 해당하는 특권 비특권 나옴) 특권 및 비특권 그룹 설정
    pri_group = {}; nonpri_group = {}
    for i in protect_variable:
        특권,비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_each, i,label_enc_recod,valid = True,print_ =False)
        pri_group.update(특권)
        nonpri_group.update(비특권)
        # 이게 이전 결과에 대한 eod와 di를 보고 그룹을 나눈거고
        
    nonpri_group = [{key: value} for key, values in nonpri_group.items() for value in values]
    pri_group = [{key: value} for key, values in pri_group.items() for value in values]
    
    nonpri_group_each = [s for s in nonpri_group if any(protect_var in key for key in s.keys())]
    pri_group_each = [s for s in pri_group if any(protect_var in key for key in s.keys())]
    # protect_var이 '직업_group'인경우 nonpri_group_each랑 pri_group_each에는 직업에 대한 특권 및 비특권 나눠짐.rw에 파라미터로 들어갈 수 잇는 형태로 
    # 여기서 이번에 rw에 적용할 보호변수 종류 하나를 여기서 뽑고 여기서 그 보호변수으 ㅣ특권과 비특권 범주가 저장이 됨
    print('비특권:',nonpri_group_each)
    print('특권:',pri_group_each)
    # protect_var에 대해서 Reweighing 적용
    RW = Reweighing(unprivileged_groups=nonpri_group_each, privileged_groups=pri_group_each)
    train_dataset_rw = RW.fit_transform(train_dataset)
    # 위에서 보호변수 중에 하나 뽑아서 그 변수의 범주들을 특권 비특권 나눠진걸 데이터 가중치 조절 rw써서 전처리 수행한 데이터셋이 이거
    
    # 학습 데이터 추출
    X_train_rw = X_train.copy()  # 보호변수를 제외한 학습 데이터 사용
    y_train_rw = train_dataset_rw.labels.ravel()
    sample_weights = train_dataset_rw.instance_weights

    best_params = best_param

    # 최적 하이퍼파라미터로 모델 학습
    model_xgb_rw = xgb.XGBClassifier(random_state=42, **best_params)
    model_xgb_rw.fit(X_train_rw, y_train_rw, sample_weight=sample_weights) # protect_var에 대해서만 rw한 데이터로 학습및 예측

    # 평가
    y_pred_rw_valid = model_xgb_rw.predict(X_valid)
    y_pred_rw_test = model_xgb_rw.predict(X_test)

    print(f"\n=== {protect_var}에 대한 Reweighing 적용 후 성능 평가 ===")
    get_clf_eval(y_valid, y_pred_rw_valid)
    
    # valid_dataset_pred_no_rw = BinaryLabelDataset(df=pd.concat([valid_df[protect_variable], pd.DataFrame(valid_dataset_pred_rw_each, columns=['AKI'])], axis=1), 
    #                                             label_names=['AKI'], 
    #                                             protected_attribute_names=protect_variable)


    valid_dataset_pred_rw = BinaryLabelDataset(
        df=pd.concat([valid_df[protect_variable], pd.DataFrame(y_pred_rw_valid, columns=['AKI'])], axis=1),
                                            label_names=['AKI'],
                                            protected_attribute_names=protect_variable)

    for i in protect_variable:
        calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw, i,label_enc_recod)
   

    return y_pred_rw_valid, y_pred_rw_test, best_params, train_dataset_rw


In [121]:
label_enc_recod

{'성별': array([0, 1]),
 '직업_group': array(['군인', '기타', '무직', '자유업', '전문직', '주부', '학생', '회사원'], dtype=object),
 '종교_group': array(['개신교', '기타', '무교', '불교', '천주교'], dtype=object),
 '연령대': array(['20.0대', '30.0대', '40.0대', '50.0대', '60.0대', '70.0대', '80.0대',
        '90.0대'], dtype=object),
 '수진일 당시 결혼상태': array(['0', '기혼', '동거', '미혼', '별거', '사별', '이혼'], dtype=object),
 'department_group_bigggest': array(['기타 진료과', '내과 계열', '외과 계열', '응급의학과'], dtype=object)}

- BinaryLabelDataset으로 변환( AIF360에서 공정성 지표를 계산하기 위한 표준 데이터 형식)

In [122]:
protect_variable

['성별',
 '직업_group',
 '종교_group',
 '연령대',
 '수진일 당시 결혼상태',
 'department_group_bigggest']

In [123]:
from aif360.datasets import BinaryLabelDataset

# BinaryLabelDataset으로 변환
train_dataset = BinaryLabelDataset(df=pd.concat([train_df[protect_variable].reset_index(drop = True), y_train], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=protect_variable)

valid_dataset = BinaryLabelDataset(df=pd.concat([valid_df[protect_variable].reset_index(drop = True), y_valid], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=protect_variable)

test_dataset = BinaryLabelDataset(df=pd.concat([test_df[protect_variable].reset_index(drop = True), y_test], axis=1), 
                                  label_names=['AKI'], 
                                  protected_attribute_names=protect_variable)


In [124]:
print(test_dataset.feature_names)

['성별', '직업_group', '종교_group', '연령대', '수진일 당시 결혼상태', 'department_group_bigggest']


# 여기서부터 진짜 모델링

## xgb & 공정성까지

### xgb


In [ ]:
import optuna
# from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# 양성 클래스와 음성 클래스의 비율 계산
ratio =(len(y_train) - sum(y_train)) / sum(y_train) # 24.34
# Optuna를 위한 목적 함수
def objective(trial):
# precision 높이려고 임계값을 애초에 낮춰서 튜닝

    param = {
        'random_state': 42,
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', ratio-5, ratio+5),
        'n_estimators': trial.suggest_int('n_estimators', 300, 500),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
        'gamma': trial.suggest_loguniform('gamma', 1e-6, 1e-1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'n_jobs':-1
    }


    # 모델 생성 및 학습
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # 확률 예측
    y_pred_prob = model.predict_proba(X_valid)[:, 1]  # 양성 클래스에 대한 확률만 가져옴

    # 임계값을 0.4로 설정
    threshold = 0.5
    y_pred = (y_pred_prob >= threshold).astype(int)

    # 평가 (recall로 최적화)
    score = recall_score(y_valid, y_pred, average='binary')
    precision = precision_score(y_valid, y_pred, average='binary')
    print(f" Best Recall: {score}, Best Precision: {precision}")
    
    return score

# Optuna Study 생성 및 최적화
sampler = optuna.samplers.TPESampler(seed=42)  # 여기서 시드 설정
study = optuna.create_study(direction='maximize',sampler=sampler)
# study = optuna.create_study(direction='maximize',sampler=sampler)
study.optimize(objective, n_trials=60)

# 최적화 결과 출력
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [74]:
recall_76 = {'scale_pos_weight': 34.038946177074195, 'n_estimators': 597, 'alpha': 0.002466653209468337, 'lambda': 0.00801693049130808, 'gamma': 0.00017877011975261608, 'learning_rate': 0.010812944394040205, 'max_depth': 6}
recall_76 = {'scale_pos_weight': 29.289012787316707, 'n_estimators': 379, 'alpha': 0.0003178865010591518, 'gamma': 5.563311760304072e-05, 'learning_rate': 0.09670638181366002, 'max_depth': 3}
best_param = {'scale_pos_weight': 21.78671977652665, 'n_estimators': 493, 'alpha': 0.0002207134419313121, 'gamma': 0.008285871695517246, 'learning_rate': 0.02416051184582963, 'max_depth': 5}
best_param = {'scale_pos_weight': 25.67897312210368, 'n_estimators': 490, 'alpha': 1.677701328392082e-05, 'gamma': 0.0821054934353308, 'learning_rate': 0.023017620614473464, 'max_depth': 6}

-  Reweighing 적용 전 모델 학습

In [ ]:
model_xgb_no_rw  = xgb.XGBClassifier(random_state=42,**best_param) 

model_xgb_no_rw.fit(X_train, y_train)

XGBClassifier(alpha=1.677701328392082e-05, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0821054934353308, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.023017620614473464,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=490, n_jobs=None,
              num_parallel_tree=None, ...)

In [76]:
# Reweighing 적용 전 평가
print("=== Reweighing 적용 전 성능 평가 ===")
y_pred_no_rw_valid  = model_xgb_no_rw.predict(X_valid)
get_clf_eval(y_valid, y_pred_no_rw_valid)
# 정밀도 0.2 목표
# 재현율 0.75 목표

=== Reweighing 적용 전 성능 평가 ===
오차행렬:
 [[13928  1949]
 [  185   455]]

정확도: 0.8708
정밀도(precision 0.2 목표): 0.1893
재현율(recall 0.75 목표): 0.7109
F1: 0.2989


In [77]:
# Reweighing 적용 전 평가
print("=== Reweighing 적용 전 성능 평가 ===")
y_pred_no_rw_test  = model_xgb_no_rw.predict(X_test)
get_clf_eval(y_test, y_pred_no_rw_test)
# 정밀도 0.2 목표
# 재현율 0.75 목표

=== Reweighing 적용 전 성능 평가 ===
오차행렬:
 [[15414  2182]
 [  182   526]]

정확도: 0.8708
정밀도(precision 0.2 목표): 0.1942
재현율(recall 0.75 목표): 0.7429
F1: 0.3080


In [78]:
protect_variable

['성별',
 '직업_group',
 '종교_group',
 '연령대',
 '수진일 당시 결혼상태',
 'department_group_bigggest']

In [79]:
test_dataset_pred_no_rw = BinaryLabelDataset(
    df=pd.concat([test_df[protect_variable].reset_index(drop = True), pd.DataFrame(y_pred_no_rw_test, columns=['AKI'])], axis=1), 
                                             label_names=['AKI'], 
                                             protected_attribute_names=protect_variable)

valid_dataset_pred_no_rw = BinaryLabelDataset(
    df=pd.concat([valid_df[protect_variable].reset_index(drop = True), pd.DataFrame(y_pred_no_rw_valid, columns=['AKI'])], axis=1), 
                                             label_names=['AKI'], 
                                             protected_attribute_names=protect_variable)
print("=== Reweighing 적용 전 성능 평가 ===")
get_clf_eval(y_valid, y_pred_no_rw_valid)
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권,비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, i,label_enc_recod,True)
    pri_group.update(특권)
    nonpri_group.update(비특권)

=== Reweighing 적용 전 성능 평가 ===
오차행렬:
 [[13928  1949]
 [  185   455]]

정확도: 0.8708
정밀도(precision 0.2 목표): 0.1893
재현율(recall 0.75 목표): 0.7109
F1: 0.2989

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 1.3701482888601788
(1과의 거리:0.3701482888601788)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 0.7298480085187685
(1과의 거리:0.2701519914812315)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.1410516813360415
(0과의 거리:0.1410516813360415) 

DI: 0.9908997971471809
(1과의 거리:0.009100202852819095)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.01858372456964008
(0과의 거리:0.01858372456964008) 

DI: 0.9838037066686716
(1과의 거리:0.016196293331328437)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.009820867379006892
(0과의 거리:0.009820867379006892) 

DI: 1.5061491926320285
(1과의 거리:0.5061491926320285)


=== 직업_group: 자유업 기준 공정성 평가 ===
EOD: -0.08711745057325992
(0과의 거리:0.08711745057325992) 

DI: 0.8485617975608664
(1과의 거리:0.15143820243913364)


=== 직

In [82]:
label_enc_recod

{'성별': array([0, 1]),
 '직업_group': array(['군인', '기타', '무직', '자유업', '전문직', '주부', '학생', '회사원'], dtype=object),
 '종교_group': array(['개신교', '기타', '무교', '불교', '천주교'], dtype=object),
 '연령대': array(['20.0대', '30.0대', '40.0대', '50.0대', '60.0대', '70.0대', '80.0대',
        '90.0대'], dtype=object),
 '수진일 당시 결혼상태': array(['0', '기혼', '동거', '미혼', '별거', '사별', '이혼'], dtype=object),
 'department_group_bigggest': array(['기타 진료과', '내과 계열', '외과 계열', '응급의학과'], dtype=object)}

In [83]:
pri_group

{'성별': [0],
 '직업_group': [0, 1, 2],
 '종교_group': [0, 1, 4],
 '연령대': [5, 6, 7],
 '수진일 당시 결혼상태': [0, 1, 4, 5],
 'department_group_bigggest': [1]}

# 함수써보자...ㄷㄱ

### 직업 보호변수에 대해 reweighing 적용 및 성능 평가
- 성별로 Reweighing 후 나온 결과를 기반으로 직업_group 보호변수에 대해 Reweighing 적용

### 성별 보호변수에 대해 reweighing 적용 및 성능 평가

In [84]:
# 성별에 대해 Reweighing 적용 및 성능 평가
y_pred_rw_valid_gender, y_pred_rw_test_gender, best_params_gender, train_dataset_rw_gender = apply_reweighing_tuning(
    train_dataset=train_dataset, 
    valid_dataset=valid_dataset,
    valid_dataset_pred_rw_each=valid_dataset_pred_no_rw, 
    protect_var='성별',  # 성별에 대해 RW 적용
    best_param=best_param, 
    X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
)
print("\n=== 성별에 대한 Reweighing 적용 후 성능 평가 ===")
get_clf_eval(y_valid, y_pred_rw_valid_gender)
# get_clf_eval(y_test, y_pred_rw_test_gender)

# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
valid_dataset_pred_rw_gender = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
valid_dataset_pred_rw_gender.labels = y_pred_rw_valid_gender  # 예측된 라벨로 대체

pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권,비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_gender, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)

비특권: [{'성별': 1}]
특권: [{'성별': 0}]

=== 성별에 대한 Reweighing 적용 후 성능 평가 ===
오차행렬:
 [[13947  1930]
 [  182   458]]

정확도: 0.8721
정밀도(precision 0.2 목표): 0.1918
재현율(recall 0.75 목표): 0.7156
F1: 0.3025

=== 성별: 0 기준 공정성 평가 ===
EOD: -0.003456221198156695
(0과의 거리:0.003456221198156695) 

DI: 1.2610828902481401
(1과의 거리:0.26108289024814013)


=== 성별: 1 기준 공정성 평가 ===
EOD: 0.003456221198156695
(0과의 거리:0.003456221198156695) 

DI: 0.7929692867399323
(1과의 거리:0.20703071326006772)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.14579101782893256
(0과의 거리:0.14579101782893256) 

DI: 0.9975809588641447
(1과의 거리:0.0024190411358553376)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.0238654147104852
(0과의 거리:0.0238654147104852) 

DI: 0.9910743114292715
(1과의 거리:0.008925688570728507)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.02624135763670643
(0과의 거리:0.02624135763670643) 

DI: 1.4860839244537503
(1과의 거리:0.48608392445375026)


=== 직업_group: 자유업 기준 공정성 평가 ===
EOD: -0.07300412265655565
(0과의 거리:0.07300412265655565) 

DI: 0.80984096696436

In [85]:
# Reweighing 전
print("Reweighing 전 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, '성별', label_enc_recod)

# Reweighing 후
print("\n\nReweighing 후 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_gender, '성별', label_enc_recod)


Reweighing 전 EOD/DI 점수:

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 1.3701482888601788
(1과의 거리:0.3701482888601788)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 0.7298480085187685
(1과의 거리:0.2701519914812315)



Reweighing 후 EOD/DI 점수:

=== 성별: 0 기준 공정성 평가 ===
EOD: -0.003456221198156695
(0과의 거리:0.003456221198156695) 

DI: 1.2610828902481401
(1과의 거리:0.26108289024814013)


=== 성별: 1 기준 공정성 평가 ===
EOD: 0.003456221198156695
(0과의 거리:0.003456221198156695) 

DI: 0.7929692867399323
(1과의 거리:0.20703071326006772)



({}, {})

### 직업 보호변수에 대해 reweighing 적용 및 성능 평가
- 성별로 Reweighing 후 나온 결과를 기반으로 직업_group 보호변수에 대해 Reweighing 적용

In [86]:
# 성별에 대해 Reweighing 적용 후 직업에 대해 추가로 Reweighing 적용
y_pred_rw_valid_job, y_pred_rw_test_job, best_params_job, train_dataset_rw_job = apply_reweighing_tuning(
    train_dataset=train_dataset_rw_gender,  # 성별 RW 적용된 데이터셋을 사용
    valid_dataset=valid_dataset,
    valid_dataset_pred_rw_each=valid_dataset_pred_rw_gender,  # 성별 RW 적용된 검증 결과
    protect_var='직업_group',  # 직업에 대해 RW 적용
    best_param=best_param,  # 동일한 best_param 사용
    X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
)

# 직업까지 Reweighing 적용 후 최종 성능 평가
print(f"\n=== 직업에 대해 추가 Reweighing 적용 후 성능 평가 ===")
get_clf_eval(y_valid, y_pred_rw_valid_job)

# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
valid_dataset_pred_rw_job = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
valid_dataset_pred_rw_job.labels = y_pred_rw_valid_job  # 예측된 라벨로 대체

pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권,비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_job, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)

비특권: [{'직업_group': 3}, {'직업_group': 4}, {'직업_group': 5}, {'직업_group': 6}, {'직업_group': 7}]
특권: [{'직업_group': 0}, {'직업_group': 1}, {'직업_group': 2}]

=== 직업_group에 대한 Reweighing 적용 후 성능 평가 ===
오차행렬:
 [[13947  1930]
 [  185   455]]

정확도: 0.8720
정밀도(precision 0.2 목표): 0.1908
재현율(recall 0.75 목표): 0.7109
F1: 0.3008

=== 성별: 0 기준 공정성 평가 ===
EOD: -0.017692560895325893
(0과의 거리:0.017692560895325893) 

DI: 1.2238287034825395
(1과의 거리:0.22382870348253947)


=== 성별: 1 기준 공정성 평가 ===
EOD: 0.017692560895325893
(0과의 거리:0.017692560895325893) 

DI: 0.8171078167674853
(1과의 거리:0.18289218323251466)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.1410516813360415
(0과의 거리:0.1410516813360415) 

DI: 0.9318609479933815
(1과의 거리:0.06813905200661852)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.01858372456964008
(0과의 거리:0.01858372456964008) 

DI: 1.0022775991425508
(1과의 거리:0.002277599142550768)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.009820867379006892
(0과의 거리:0.009820867379006892) 

DI: 1.465018502153982
(1과의 거리:0.46501850215

In [88]:
# Reweighing 전
print("Reweighing 전 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, '직업_group', label_enc_recod)

print('------------------------------------------------------------')
# Reweighing 후
print("\n\nReweighing 후 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_job, '직업_group', label_enc_recod)


Reweighing 전 EOD/DI 점수:

=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.1410516813360415
(0과의 거리:0.1410516813360415) 

DI: 0.9908997971471809
(1과의 거리:0.009100202852819095)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.01858372456964008
(0과의 거리:0.01858372456964008) 

DI: 0.9838037066686716
(1과의 거리:0.016196293331328437)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.009820867379006892
(0과의 거리:0.009820867379006892) 

DI: 1.5061491926320285
(1과의 거리:0.5061491926320285)


=== 직업_group: 자유업 기준 공정성 평가 ===
EOD: -0.08711745057325992
(0과의 거리:0.08711745057325992) 

DI: 0.8485617975608664
(1과의 거리:0.15143820243913364)


=== 직업_group: 전문직 기준 공정성 평가 ===
EOD: 0.020671510899523904
(0과의 거리:0.020671510899523904) 

DI: 0.9008137735270415
(1과의 거리:0.0991862264729585)


=== 직업_group: 주부 기준 공정성 평가 ===
EOD: 0.03937801212406178
(0과의 거리:0.03937801212406178) 

DI: 0.7082247248197623
(1과의 거리:0.2917752751802377)


=== 직업_group: 학생 기준 공정성 평가 ===
EOD: -0.21159874608150475
(0과의 거리:0.21159874608150475) 

DI: 0.25906082072613257
(1과의 거리:0.7

({}, {})

### 종교에 대해 공정성 개선
- 종교는 공정성 개선이 필요없을 정도로 값이 좋다고 생각이 들어서 적용하지 않겠음

In [29]:
# # 종교에 대해 추가로 Reweighing 적용
# y_pred_rw_valid_religion, y_pred_rw_test_religion, best_params_religion, train_dataset_rw_religion = apply_reweighing_tuning(
#     train_dataset=train_dataset_rw_job, 
#     valid_dataset=valid_dataset,
#     valid_dataset_pred_rw_each=valid_dataset_pred_rw_job,  # 성별 RW 적용된 검증 결과
#     protect_var='종교_group',  # 종교에 대해 RW 적용
#     best_param=best_param,  # 동일한 best_param 사용
#     X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
# )

# # 종교까지 Reweighing 적용 후 최종 성능 평가
# print(f"\n=== 종교에 대해 추가 Reweighing 적용 후 성능 평가 ===")
# get_clf_eval(y_valid, y_pred_rw_valid_religion)

# # Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
# valid_dataset_pred_rw_religion = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
# valid_dataset_pred_rw_religion.labels = y_pred_rw_valid_religion  # 예측된 라벨로 대체

# # 공정성 지표(EOD/DI)를 종교 기준으로 계산
# pri_group = {}; nonpri_group = {}
# for i in protect_variable:
#     특권, 비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_religion, i, label_enc_recod, True)
#     pri_group.update(특권)
#     nonpri_group.update(비특권)

### 위 결과 종교는 나쁘지 않아보여서 연령대로 rw

In [89]:
print("\n=== 연령대 보호변수에 대한 Reweighing 적용 및 성능 평가 ===")
y_pred_rw_valid_age, y_pred_rw_test_age, best_params_age ,train_dataset_age = apply_reweighing_tuning(
    train_dataset=train_dataset_rw_job,  # 직업까지 RW 적용된 데이터셋을 사용
    valid_dataset=valid_dataset,
    valid_dataset_pred_rw_each=valid_dataset_pred_rw_job,  # 직업 RW 적용된 검증 결과
    protect_var='연령대',  # 연령대에 대해 RW 적용
    best_param=best_param,  # 동일한 best_param 사용
    X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
)

# 연령대까지 Reweighing 적용 후 최종 성능 평가
print(f"\n=== 연령대에 대해 추가 Reweighing 적용 후 성능 평가 ===")
get_clf_eval(y_valid, y_pred_rw_valid_age)


# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
valid_dataset_pred_rw_age = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
valid_dataset_pred_rw_age.labels = y_pred_rw_valid_age  # 예측된 라벨로 대체

# 공정성 지표(EOD/DI)를 연령대 기준으로 계산
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권, 비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_age, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)


=== 연령대 보호변수에 대한 Reweighing 적용 및 성능 평가 ===
비특권: [{'연령대': 0}, {'연령대': 1}, {'연령대': 2}, {'연령대': 3}, {'연령대': 4}]
특권: [{'연령대': 5}, {'연령대': 6}, {'연령대': 7}]

=== 연령대에 대한 Reweighing 적용 후 성능 평가 ===
오차행렬:
 [[13994  1883]
 [  187   453]]

정확도: 0.8747
정밀도(precision 0.2 목표): 0.1939
재현율(recall 0.75 목표): 0.7078
F1: 0.3044

=== 성별: 0 기준 공정성 평가 ===
EOD: -0.003044766293614276
(0과의 거리:0.003044766293614276) 

DI: 1.239397814780356
(1과의 거리:0.23939781478035593)


=== 성별: 1 기준 공정성 평가 ===
EOD: 0.003044766293614276
(0과의 거리:0.003044766293614276) 

DI: 0.8068434428999041
(1과의 거리:0.19315655710009594)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: 0.006544798013992326
(0과의 거리:0.006544798013992326) 

DI: 1.0883952254641909
(1과의 거리:0.08839522546419087)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.03071205007824729
(0과의 거리:0.03071205007824729) 

DI: 1.0356393286161438
(1과의 거리:0.03563932861614383)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.028205531112507898
(0과의 거리:0.028205531112507898) 

DI: 1.4404837135289301
(1과의 거리:0.4404837135

In [90]:
# Reweighing 전
print("Reweighing 전 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, '연령대', label_enc_recod)

print('------------------------------------------------------------')
# Reweighing 후
print("\n\nReweighing 후 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_age, '연령대', label_enc_recod)


Reweighing 전 EOD/DI 점수:

=== 연령대: 20.0대 기준 공정성 평가 ===
EOD: -0.42992552471225454
(0과의 거리:0.42992552471225454) 

DI: 0.4485818903990566
(1과의 거리:0.5514181096009434)


=== 연령대: 30.0대 기준 공정성 평가 ===
EOD: -0.02754470717857449
(0과의 거리:0.02754470717857449) 

DI: 0.8421528205034023
(1과의 거리:0.15784717949659766)


=== 연령대: 40.0대 기준 공정성 평가 ===
EOD: -0.055987621645832486
(0과의 거리:0.055987621645832486) 

DI: 0.7008734091351269
(1과의 거리:0.29912659086487314)


=== 연령대: 50.0대 기준 공정성 평가 ===
EOD: 0.04825636608582351
(0과의 거리:0.04825636608582351) 

DI: 0.7726663190587947
(1과의 거리:0.22733368094120532)


=== 연령대: 60.0대 기준 공정성 평가 ===
EOD: 0.05135090034739609
(0과의 거리:0.05135090034739609) 

DI: 0.8354168988767035
(1과의 거리:0.16458310112329655)


=== 연령대: 70.0대 기준 공정성 평가 ===
EOD: 0.01152900184464023
(0과의 거리:0.01152900184464023) 

DI: 1.0678746127384677
(1과의 거리:0.06787461273846773)


=== 연령대: 80.0대 기준 공정성 평가 ===
EOD: -0.042168438394853514
(0과의 거리:0.042168438394853514) 

DI: 1.409130477962571
(1과의 거리:0.40913047796257107

({}, {})

### 수진일 당시 결혼상태로 rw

In [91]:
# 결혼상태 보호변수에 대해 추가 Reweighing 적용
print("\n=== 결혼상태 보호변수에 대한 Reweighing 적용 및 성능 평가 ===")
y_pred_rw_valid_marriage, y_pred_rw_test_marriage, best_params_marriage, train_dataset_rw_marriage = apply_reweighing_tuning(
    train_dataset=train_dataset_age,  # 연령대까지 RW 적용된 데이터셋을 사용
    valid_dataset=valid_dataset,
    valid_dataset_pred_rw_each=valid_dataset_pred_rw_age,  # 연령대 RW 적용된 검증 결과
    protect_var='수진일 당시 결혼상태',  # 결혼상태에 대해 RW 적용
    best_param=best_param,  # 동일한 best_param 사용
    X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
)

# 결혼상태까지 Reweighing 적용 후 최종 성능 평가
print(f"\n=== 결혼상태에 대해 추가 Reweighing 적용 후 성능 평가 ===")
get_clf_eval(y_valid, y_pred_rw_valid_marriage)

# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
valid_dataset_pred_rw_marriage = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
valid_dataset_pred_rw_marriage.labels = y_pred_rw_valid_marriage  # 예측된 라벨로 대체

# 공정성 지표(EOD/DI)를 결혼상태 기준으로 계산
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권, 비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_marriage, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)


=== 결혼상태 보호변수에 대한 Reweighing 적용 및 성능 평가 ===
비특권: [{'수진일 당시 결혼상태': 1}, {'수진일 당시 결혼상태': 2}, {'수진일 당시 결혼상태': 3}, {'수진일 당시 결혼상태': 6}]
특권: [{'수진일 당시 결혼상태': 0}, {'수진일 당시 결혼상태': 4}, {'수진일 당시 결혼상태': 5}]

=== 수진일 당시 결혼상태에 대한 Reweighing 적용 후 성능 평가 ===
오차행렬:
 [[13984  1893]
 [  189   451]]

정확도: 0.8739
정밀도(precision 0.2 목표): 0.1924
재현율(recall 0.75 목표): 0.7047
F1: 0.3023

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.005019749835417975
(0과의 거리:0.005019749835417975) 

DI: 1.2802399818685914
(1과의 거리:0.28023998186859145)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.005019749835417975
(0과의 거리:0.005019749835417975) 

DI: 0.7811035541480563
(1과의 거리:0.21889644585194368)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: 0.009704355675919651
(0과의 거리:0.009704355675919651) 

DI: 1.0846550356859637
(1과의 거리:0.08465503568596366)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.01154147104851333
(0과의 거리:0.01154147104851333) 

DI: 1.046844341961567
(1과의 거리:0.04684434196156695)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.015163419233186648
(0과의 거리:0.015163419233

In [92]:
# Reweighing 전
print("Reweighing 전 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, '수진일 당시 결혼상태', label_enc_recod)

print('------------------------------------------------------------')
# Reweighing 후
print("\n\nReweighing 후 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_marriage, '수진일 당시 결혼상태', label_enc_recod)


Reweighing 전 EOD/DI 점수:

=== 수진일 당시 결혼상태: 0 기준 공정성 평가 ===
EOD: 0.07644910999543586
(0과의 거리:0.07644910999543586) 

DI: 1.123795711177872
(1과의 거리:0.12379571117787203)


=== 수진일 당시 결혼상태: 기혼 기준 공정성 평가 ===
EOD: -0.02622377622377614
(0과의 거리:0.02622377622377614) 

DI: 0.9831162613679435
(1과의 거리:0.0168837386320565)


=== 수진일 당시 결혼상태: 동거 기준 공정성 평가 ===
EOD: 0.28996865203761757
(0과의 거리:0.28996865203761757) 

DI: 1.0852622777679986
(1과의 거리:0.08526227776799855)


=== 수진일 당시 결혼상태: 미혼 기준 공정성 평가 ===
EOD: -0.05756578947368418
(0과의 거리:0.05756578947368418) 

DI: 0.7494371023005384
(1과의 거리:0.2505628976994616)


=== 수진일 당시 결혼상태: 별거 기준 공정성 평가 ===
EOD: 0.0897637795275591
(0과의 거리:0.0897637795275591) 

DI: 1.1455898720089037
(1과의 거리:0.14558987200890372)


=== 수진일 당시 결혼상태: 사별 기준 공정성 평가 ===
EOD: -0.019395866454690047
(0과의 거리:0.019395866454690047) 

DI: 1.157768220839438
(1과의 거리:0.15776822083943798)


=== 수진일 당시 결혼상태: 이혼 기준 공정성 평가 ===
EOD: 0.2402643801357628
(0과의 거리:0.2402643801357628) 

DI: 1.0323726412173706
(1

({}, {})

### department_group_bigggest로 rw

In [93]:
print("\n=== department_group_bigggest에 대한 Reweighing 적용 및 성능 평가 ===")
y_pred_rw_valid_department, y_pred_rw_test_department, best_params_department, train_dataset_rw_department = apply_reweighing_tuning(
    train_dataset=train_dataset_rw_marriage,  # 결혼상태까지 RW 적용된 데이터셋을 사용
    valid_dataset=valid_dataset,
    valid_dataset_pred_rw_each=valid_dataset_pred_rw_marriage,  # 결혼상태 RW 적용된 검증 결과
    protect_var='department_group_bigggest',  # department_group_bigggest에 대해 RW 적용
    best_param=best_param,  # 동일한 best_param 사용
    X_train=X_train, X_valid=X_valid, X_test=X_test, y_valid=y_valid
)

# department_group_bigggest까지 Reweighing 적용 후 최종 성능 평가
print(f"\n=== department_group_bigggest에 대해 추가 Reweighing 적용 후 성능 평가 ===")
get_clf_eval(y_valid, y_pred_rw_valid_department)

# Reweighing 후 예측 결과를 BinaryLabelDataset으로 변환
valid_dataset_pred_rw_department = valid_dataset.copy(deepcopy=True)  # 원래 dataset 구조를 복사
valid_dataset_pred_rw_department.labels = y_pred_rw_valid_department  # 예측된 라벨로 대체

# 공정성 지표(EOD/DI)를 department_group_bigggest 기준으로 계산
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권, 비특권 = calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_department, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)


=== department_group_bigggest에 대한 Reweighing 적용 및 성능 평가 ===
비특권: [{'department_group_bigggest': 0}, {'department_group_bigggest': 2}, {'department_group_bigggest': 3}]
특권: [{'department_group_bigggest': 1}]

=== department_group_bigggest에 대한 Reweighing 적용 후 성능 평가 ===
오차행렬:
 [[14011  1866]
 [  185   455]]

정확도: 0.8758
정밀도(precision 0.2 목표): 0.1960
재현율(recall 0.75 목표): 0.7109
F1: 0.3073

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 1.2617398723757107
(1과의 거리:0.2617398723757107)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.0020572745227123157
(0과의 거리:0.0020572745227123157) 

DI: 0.7925563912925374
(1과의 거리:0.2074436087074626)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: 0.003385240352065
(0과의 거리:0.003385240352065) 

DI: 1.095478057734023
(1과의 거리:0.09547805773402307)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.04988262910798125
(0과의 거리:0.04988262910798125) 

DI: 0.9974163383193202
(1과의 거리:0.002583661680679805)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.016106222501571343

In [94]:
# Reweighing 전
print("Reweighing 전 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_no_rw, 'department_group_bigggest', label_enc_recod)

print('------------------------------------------------------------')
# Reweighing 후
print("\n\nReweighing 후 EOD/DI 점수:")
calculate_eod_di_by_category(valid_dataset, valid_dataset_pred_rw_department, 'department_group_bigggest', label_enc_recod)


Reweighing 전 EOD/DI 점수:

=== department_group_bigggest: 기타 진료과 기준 공정성 평가 ===
EOD: -0.044689800210305
(0과의 거리:0.044689800210305) 

DI: 0.572348992180551
(1과의 거리:0.427651007819449)


=== department_group_bigggest: 내과 계열 기준 공정성 평가 ===
EOD: 0.11526165236412578
(0과의 거리:0.11526165236412578) 

DI: 1.6222742047025829
(1과의 거리:0.6222742047025829)


=== department_group_bigggest: 외과 계열 기준 공정성 평가 ===
EOD: -0.15662211283173244
(0과의 거리:0.15662211283173244) 

DI: 0.5905681571499372
(1과의 거리:0.40943184285006284)


=== department_group_bigggest: 응급의학과 기준 공정성 평가 ===
EOD: 0.05178392189068348
(0과의 거리:0.05178392189068348) 

DI: 1.0485197628516838
(1과의 거리:0.0485197628516838)

------------------------------------------------------------


Reweighing 후 EOD/DI 점수:

=== department_group_bigggest: 기타 진료과 기준 공정성 평가 ===
EOD: 0.12355415352260779
(0과의 거리:0.12355415352260779) 

DI: 0.663304130484094
(1과의 거리:0.336695869515906)


=== department_group_bigggest: 내과 계열 기준 공정성 평가 ===
EOD: 0.07091882689471551
(0과의 거리:0.07091

({}, {})

#### test 데이터에 대해 rw 적용전 공정성 점수와 rw 적용 후 공정성 점수

In [95]:
# Reweighing 전 test 데이터셋에 대한 성능 평가
print("=== Reweighing 적용 전 test 데이터셋 성능 평가 ===")
get_clf_eval(y_test, y_pred_no_rw_test)

# Reweighing 후 test 데이터셋에 대한 성능 평가
print("\n=== Reweighing 적용 후 test 데이터셋 성능 평가 ===")
get_clf_eval(y_test, y_pred_rw_test_department)  # Reweighing 적용 후 예측


=== Reweighing 적용 전 test 데이터셋 성능 평가 ===
오차행렬:
 [[15414  2182]
 [  182   526]]

정확도: 0.8708
정밀도(precision 0.2 목표): 0.1942
재현율(recall 0.75 목표): 0.7429
F1: 0.3080

=== Reweighing 적용 후 test 데이터셋 성능 평가 ===
오차행렬:
 [[15479  2117]
 [  185   523]]

정확도: 0.8742
정밀도(precision 0.2 목표): 0.1981
재현율(recall 0.75 목표): 0.7387
F1: 0.3124


In [96]:
# Reweighing 전 test 데이터셋에 대한 공정성 지표 계산 (EOD/DI)
print("\n=== Reweighing 적용 전 test 데이터셋 EOD/DI 점수 ===")
test_dataset_pred_test_no_rw = test_dataset.copy(deepcopy=True)
test_dataset_pred_test_no_rw.labels = y_pred_no_rw_test  # 예측 결과로 대체
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권, 비특권 = calculate_eod_di_by_category(test_dataset, test_dataset_pred_test_no_rw, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)


=== Reweighing 적용 전 test 데이터셋 EOD/DI 점수 ===

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.042826738478912385
(0과의 거리:0.042826738478912385) 

DI: 1.4143458227239056
(1과의 거리:0.41434582272390563)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.042826738478912385
(0과의 거리:0.042826738478912385) 

DI: 0.7070406571952028
(1과의 거리:0.2929593428047972)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.24362606232294615
(0과의 거리:0.24362606232294615) 

DI: 0.918308214586892
(1과의 거리:0.08169178541310795)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.052391644553310246
(0과의 거리:0.052391644553310246) 

DI: 0.8860182044707587
(1과의 거리:0.11398179552924126)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.03551731568907579
(0과의 거리:0.03551731568907579) 

DI: 1.5816550255091615
(1과의 거리:0.5816550255091615)


=== 직업_group: 자유업 기준 공정성 평가 ===
EOD: -0.03880846991266895
(0과의 거리:0.03880846991266895) 

DI: 0.854952285458408
(1과의 거리:0.14504771454159204)


=== 직업_group: 전문직 기준 공정성 평가 ===
EOD: -0.05256034288292355
(0과의 거리:0.05256034288292355) 

DI: 0.6758724884807072
(1과

In [101]:
# Reweighing 후 test 데이터셋에 대한 공정성 지표 계산 (EOD/DI)
print("\n\n=== Reweighing 적용 후 test 데이터셋 EOD/DI 점수 ===")
test_dataset_pred_test_rw = test_dataset.copy(deepcopy=True)
test_dataset_pred_test_rw.labels = y_pred_rw_test_department  # Reweighing 적용 후 예측 결과로 대체
pri_group = {}; nonpri_group = {}
for i in protect_variable:
    특권, 비특권 = calculate_eod_di_by_category(test_dataset, test_dataset_pred_test_rw, i, label_enc_recod, True)
    pri_group.update(특권)
    nonpri_group.update(비특권)



=== Reweighing 적용 후 test 데이터셋 EOD/DI 점수 ===

=== 성별: 0 기준 공정성 평가 ===
EOD: 0.02393258914998042
(0과의 거리:0.02393258914998042) 

DI: 1.3286914471209161
(1과의 거리:0.32869144712091614)


=== 성별: 1 기준 공정성 평가 ===
EOD: -0.02393258914998042
(0과의 거리:0.02393258914998042) 

DI: 0.7526201829377743
(1과의 거리:0.24737981706222567)


=== 직업_group: 군인 기준 공정성 평가 ===
EOD: -0.23937677053824358
(0과의 거리:0.23937677053824358) 

DI: 0.8068892139912223
(1과의 거리:0.19311078600877774)


=== 직업_group: 기타 기준 공정성 평가 ===
EOD: -0.047696808872559004
(0과의 거리:0.047696808872559004) 

DI: 0.8705087208650452
(1과의 거리:0.12949127913495484)


=== 직업_group: 무직 기준 공정성 평가 ===
EOD: 0.038462459394529525
(0과의 거리:0.038462459394529525) 

DI: 1.5513177932957518
(1과의 거리:0.5513177932957518)


=== 직업_group: 자유업 기준 공정성 평가 ===
EOD: -0.051082665390597026
(0과의 거리:0.051082665390597026) 

DI: 0.8471763341647597
(1과의 거리:0.15282366583524032)


=== 직업_group: 전문직 기준 공정성 평가 ===
EOD: -0.0880893300248139
(0과의 거리:0.0880893300248139) 

DI: 0.7093969746744394
(

# 결과 만드는거

In [102]:
test_df['AKI']=y_test
test_df['예측값']=y_pred_rw_test_department
test=test_df[['등록번호', 'AKI','예측값']]
test.head()

,등록번호,AKI,예측값
0,30,0,0
1,30,0,1
2,30,0,1
3,30,0,0
4,30,0,0


In [103]:
# 1. 등록번호별 새로운 DataFrame 생성 및 'case1' ~ 'case6'을 NaN으로 초기화
unique_ids = test['등록번호'].unique()
cases_df = pd.DataFrame({
    '등록번호': unique_ids,
    'case1': np.nan,
    'case2': np.nan,
    'case3': np.nan,
    'case4': np.nan,
    'case5': np.nan,
    'case6': np.nan
})

# 2. 동일 등록번호별로 예측값에 따라 case 값 수정
for patient_id in unique_ids:
    # 해당 등록번호의 데이터 가져오기
    patient_data = test[test['등록번호'] == patient_id].reset_index(drop=True)
    
    case_updated = False  # case가 한번이라도 업데이트되면 True로 변경
    
    # 동일 등록번호 내 예측값에 따라 처리
    for idx in range(len(patient_data)):
        if not case_updated:  # 아직 case가 업데이트되지 않았을 때만 처리
            # 예측값을 해당 case에 할당
            if idx == 0:
                cases_df.loc[cases_df['등록번호'] == patient_id, 'case1'] = patient_data.loc[idx, 'AKI']
            elif idx == 1:
                cases_df.loc[cases_df['등록번호'] == patient_id, ['case1', 'case2']] = patient_data.loc[idx, 'AKI']
            elif idx == 2:
                cases_df.loc[cases_df['등록번호'] == patient_id, ['case2', 'case3']] = patient_data.loc[idx, 'AKI']
            elif idx == 3:
                cases_df.loc[cases_df['등록번호'] == patient_id, ['case3', 'case4']] = patient_data.loc[idx, 'AKI']
            elif idx == 4:
                cases_df.loc[cases_df['등록번호'] == patient_id, ['case4', 'case5']] = patient_data.loc[idx, 'AKI']
            elif idx == 5:
                cases_df.loc[cases_df['등록번호'] == patient_id, ['case5', 'case6']] = patient_data.loc[idx, 'AKI']
            elif idx == 6:
                cases_df.loc[cases_df['등록번호'] == patient_id, 'case6'] = patient_data.loc[idx, 'AKI']

            # 만약 예측값이 1이면 더 이상 업데이트하지 않도록 설정
            if patient_data.loc[idx, 'AKI'] == 1:
                case_updated = True


cases_df

,등록번호,case1,case2,case3,case4,case5,case6
0,30,0.0,0.0,0.0,0.0,0.0,0.0
1,142,0.0,0.0,0.0,0.0,0.0,0.0
2,150,0.0,0.0,0.0,0.0,0.0,0.0
3,195,0.0,0.0,0.0,0.0,0.0,0.0
4,322,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3445,229209,0.0,0.0,0.0,0.0,0.0,0.0
3446,229210,0.0,0.0,0.0,0.0,0.0,0.0
3447,229237,0.0,0.0,0.0,0.0,0.0,0.0
3448,229304,0.0,1.0,1.0,NaN,NaN,NaN


In [104]:
# 1. 등록번호별 새로운 DataFrame 생성 및 'case1' ~ 'case6'을 NaN으로 초기화
unique_ids = test['등록번호'].unique()
cases_df_pre = pd.DataFrame({
    '등록번호': unique_ids,
    'case1': np.nan,
    'case2': np.nan,
    'case3': np.nan,
    'case4': np.nan,
    'case5': np.nan,
    'case6': np.nan
})

# 2. 동일 등록번호별로 예측값에 따라 case 값 수정
for patient_id in unique_ids:
    # 해당 등록번호의 데이터 가져오기
    patient_data = test[test['등록번호'] == patient_id].reset_index(drop=True)
    
    case_updated = False  # case가 한번이라도 업데이트되면 True로 변경
    
    # 동일 등록번호 내 예측값에 따라 처리
    for idx in range(len(patient_data)):
        if not case_updated:  # 아직 case가 업데이트되지 않았을 때만 처리
            # 예측값을 해당 case에 할당
            if idx == 0:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, 'case1'] = patient_data.loc[idx, '예측값']
            elif idx == 1:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, ['case1', 'case2']] = patient_data.loc[idx, '예측값']
            elif idx == 2:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, ['case2', 'case3']] = patient_data.loc[idx, '예측값']
            elif idx == 3:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, ['case3', 'case4']] = patient_data.loc[idx, '예측값']
            elif idx == 4:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, ['case4', 'case5']] = patient_data.loc[idx, '예측값']
            elif idx == 5:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, ['case5', 'case6']] = patient_data.loc[idx, '예측값']
            elif idx == 6:
                cases_df_pre.loc[cases_df_pre['등록번호'] == patient_id, 'case6'] = patient_data.loc[idx, '예측값']

            # 만약 예측값이 1이면 더 이상 업데이트하지 않도록 설정
            if patient_data.loc[idx, '예측값'] == 1:
                case_updated = True

# 결과 출력
cases_df_pre

,등록번호,case1,case2,case3,case4,case5,case6
0,30,1.0,1.0,NaN,NaN,NaN,NaN
1,142,0.0,0.0,0.0,0.0,0.0,0.0
2,150,0.0,0.0,0.0,0.0,0.0,0.0
3,195,0.0,0.0,0.0,0.0,0.0,0.0
4,322,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3445,229209,0.0,0.0,0.0,0.0,0.0,0.0
3446,229210,1.0,1.0,NaN,NaN,NaN,NaN
3447,229237,0.0,0.0,0.0,0.0,0.0,0.0
3448,229304,0.0,1.0,1.0,NaN,NaN,NaN


In [105]:
# 각 case별로 get_clf_eval을 적용하는 for문 (NaN 값 제거 및 이진 분류 처리)
for i in range(1, 7):
    pre_case = cases_df_pre[f'case{i}']  # 예측값
    y = cases_df[f'case{i}']  # 실제값
    
    # NaN 값이 있는 행은 제외
    valid_rows = ~pre_case.isna() & ~y.isna()
    
    # 유효한 값들만 추출
    pre_case_valid = pre_case[valid_rows].astype(int)  # 이진 값으로 변환
    y_valid = y[valid_rows].astype(int)  # 이진 값으로 변환
    
    # 유효한 데이터가 있는 경우에만 평가 수행
    if len(pre_case_valid) > 0 and len(y_valid) > 0:
        print(f"Evaluating case{i}:")
        try:
            get_clf_eval(y_valid, pre_case_valid)  # 평가 함수 호출
        except ValueError as e:
            print(f"Error in case{i}: {e}")
        print("\n")  # 각 case 결과 구분을 위한 줄바꿈
    else:
        print(f"Skipping case{i} due to NaN values or no valid data.")

Evaluating case1:
오차행렬:
 [[2137  854]
 [  69  390]]

정확도: 0.7325
정밀도(precision 0.2 목표): 0.3135
재현율(recall 0.75 목표): 0.8497
F1: 0.4580


Evaluating case2:
오차행렬:
 [[1993  296]
 [  64   65]]

정확도: 0.8511
정밀도(precision 0.2 목표): 0.1801
재현율(recall 0.75 목표): 0.5039
F1: 0.2653


Evaluating case3:
오차행렬:
 [[1820  157]
 [  39   28]]

정확도: 0.9041
정밀도(precision 0.2 목표): 0.1514
재현율(recall 0.75 목표): 0.4179
F1: 0.2222


Evaluating case4:
오차행렬:
 [[1569   93]
 [  24   20]]

정확도: 0.9314
정밀도(precision 0.2 목표): 0.1770
재현율(recall 0.75 목표): 0.4545
F1: 0.2548


Evaluating case5:
오차행렬:
 [[1434   56]
 [  15   10]]

정확도: 0.9531
정밀도(precision 0.2 목표): 0.1515
재현율(recall 0.75 목표): 0.4000
F1: 0.2198


Evaluating case6:
오차행렬:
 [[1234   38]
 [   7    9]]

정확도: 0.9651
정밀도(precision 0.2 목표): 0.1915
재현율(recall 0.75 목표): 0.5625
F1: 0.2857




# XAI

In [ ]:
import lime
import lime.lime_tabular

# LIME 설명 도구 생성 (탭 형식 데이터)
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,  # LIME 설명을 위해 학습 데이터를 전달
    feature_names=X_train.columns,  # 각 특성의 이름을 전달
    class_names=['No AKI', 'AKI'],  # 타겟 클래스 이름
    mode='classification'  # 분류 모델임을 명시
)

# 테스트 데이터의 첫 번째 샘플에 대한 예측 설명
sample = train2_final[train2_final['등록번호']==229304].values  # 첫 번째 샘플 데이터

# 예측 결과 설명 생성
exp = explainer.explain_instance(
    data_row=sample,
    predict_fn=model_xgb_pre.predict_proba,  # 예측 확률을 반환하는 함수
    num_features=10  # 설명할 주요 특성의 수
)

# 설명 출력 (노트북이나 콘솔에서 시각화 가능)
exp.show_in_notebook(show_table=True)

# 시각화 출력
plt.show()

In [ ]:
# 테스트 데이터의 첫 번째 샘플에 대한 예측 설명
sample = train2_final[train2_final['등록번호']==150].values  # 첫 번째 샘플 데이터

# 예측 결과 설명 생성
exp = explainer.explain_instance(
    data_row=sample,
    predict_fn=model_xgb_pre.predict_proba,  # 예측 확률을 반환하는 함수
    num_features=10  # 설명할 주요 특성의 수
)

# 설명 출력 (노트북이나 콘솔에서 시각화 가능)
exp.show_in_notebook(show_table=True)


# 시각화 출력
plt.show()